# Preliminary EDA

In [4]:
import pandas as pd

In [16]:
data = pd.read_csv("../data/raw/world_air_quality.csv", sep=';')
data.head()

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,JP,NaN,北九州市小倉北区大門一丁目６－４８,"33.880833, 130.873056",NO,japan-soramame,ppm,0.002,2024-03-10T13:30:00+05:30,Japan
1,JP,NaN,北九州市若松区本町三丁目１３－１,"33.898056, 130.81",NO2,japan-soramame,ppm,0.005,2024-03-10T13:30:00+05:30,Japan
2,JP,NaN,北九州市門司区大里原町１２－１２,"33.895833, 130.935833",NOX,japan-soramame,ppm,0.013,2024-03-10T13:30:00+05:30,Japan
3,JP,NaN,千歳市若草４－１３,"42.786944, 141.605",NO2,japan-soramame,ppm,0.004,2024-03-10T13:30:00+05:30,Japan
4,JP,NaN,千葉市稲毛区宮野木町９９６－９,"35.653889, 140.097778",NOX,japan-soramame,ppm,0.003,2024-03-10T13:30:00+05:30,Japan


In [18]:
data = data.drop(["Country Code", "City", "Location", "Source Name"], axis=1)

data.head()

,Coordinates,Pollutant,Unit,Value,Last Updated,Country Label
0,"33.880833, 130.873056",NO,ppm,0.002,2024-03-10T13:30:00+05:30,Japan
1,"33.898056, 130.81",NO2,ppm,0.005,2024-03-10T13:30:00+05:30,Japan
2,"33.895833, 130.935833",NOX,ppm,0.013,2024-03-10T13:30:00+05:30,Japan
3,"42.786944, 141.605",NO2,ppm,0.004,2024-03-10T13:30:00+05:30,Japan
4,"35.653889, 140.097778",NOX,ppm,0.003,2024-03-10T13:30:00+05:30,Japan


In [30]:
# Create AQI score for each pollutant
import aqi

pollutants = ['NO', 'NO2', 'NOX', 'SO2', 'PM2.5', 'CO', 'O3', 'PM10', 'PM1']

for pollutant in pollutants:
    data.loc[data['Pollutant'] == pollutant, 'AQI'] = 1

data

,Coordinates,Pollutant,Unit,Value,Last Updated,Country Label,AQI
0,"33.880833, 130.873056",NO,ppm,0.002,2024-03-10T13:30:00+05:30,Japan,1.0
1,"33.898056, 130.81",NO2,ppm,0.005,2024-03-10T13:30:00+05:30,Japan,1.0
2,"33.895833, 130.935833",NOX,ppm,0.013,2024-03-10T13:30:00+05:30,Japan,1.0
3,"42.786944, 141.605",NO2,ppm,0.004,2024-03-10T13:30:00+05:30,Japan,1.0
4,"35.653889, 140.097778",NOX,ppm,0.003,2024-03-10T13:30:00+05:30,Japan,1.0
...,...,...,...,...,...,...,...
54250,"40.64389299999999, 15.872893000000001",CO,µg/m³,295.000,2024-03-11T15:30:00+05:30,Italy,1.0
54251,"46.030833, 11.905833",O3,µg/m³,35.000,2024-03-11T15:30:00+05:30,Italy,1.0
54252,"43.55472, 10.32972",CO,µg/m³,600.000,2024-03-11T15:30:00+05:30,Italy,1.0
54253,"43.91611, 11.006939999999998",PM2.5,µg/m³,15.000,2024-03-11T04:30:00+05:30,Italy,1.0


In [32]:
myaqi = aqi.to_aqi([
    (aqi.POLLUTANT_PM25, '12'),
    (aqi.POLLUTANT_PM10, '24'),
    (aqi.POLLUTANT_O3_8H, '0.087')
])
myaqi

Decimal('129')

In [21]:
# Save processed data to file
data.to_csv("../data/processed/world_air_quality.csv")